In [1]:
import numpy as np
import pandas as pd
import country_converter as coco
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

cc=coco.CountryConverter()

In [2]:
econ=pd.read_csv('Global Economy Indicators.csv')#,sep='|',encoding='cp1252'
col2=[]
# cleaning the names of the columns
for c in econ.columns:
    col2.append(c.strip())
econ.columns=col2
# cleaning the countries
for i, c in econ['Country'].items():
    econ.iloc[i,1]=c.strip()

In [3]:
# convert every name of country in standartized names with library cc
econ['Country'][econ['Country']=='D.R. of the Congo']='Democratic Republic of the Congo'
econ['Country']=cc.pandas_convert(series=econ['Country'],to='name_short',not_found=None)

C:\Users\gripin\AppData\Local\Temp\ipykernel_14548\84829938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  econ['Country'][econ['Country']=='D.R. of the Congo']='Democratic Republic of the Congo'
Former Netherlands Antilles not found in regex
USSR (Former) not found in regex
Zanzibar not found in regex
Yugoslavia (Former) not found in regex


In [4]:
# scraping of wikipedia page to get every country, region, continent in the dim geo
cont=pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_the_United_Nations_geoscheme')

In [5]:
gcont=cont[0]
colr=cont[1].columns
cont[0].columns=colr
cont[1].iloc[0,0]='Taiwan'

In [6]:
# all tables put together
gcont=pd.concat([gcont, cont[1]])

In [7]:
gcont=gcont.reset_index(drop=True)
gcont

,Country or Area,Geographical subregion,Intermediary region,Continental region,UNSD M49 Codes
0,Algeria,Northern Africa,—,Africa,012 < 015 < 002 < 001
1,Egypt,Northern Africa,—,Africa,818 < 015 < 002 < 001
2,Libya,Northern Africa,—,Africa,434 < 015 < 002 < 001
3,Morocco,Northern Africa,—,Africa,504 < 015 < 002 < 001
4,Sudan,Northern Africa,—,Africa,729 < 015 < 002 < 001
...,...,...,...,...,...
245,Tonga,Polynesia,—,Oceania,776 < 061 < 009 < 001
246,Tuvalu,Polynesia,—,Oceania,798 < 061 < 009 < 001
247,Wallis and Futuna Islands,Polynesia,—,Oceania,876 < 061 < 009 < 001
248,Taiwan,Eastern Asia,—,Asia,158 < 030 < 142 < 001


In [8]:
gcont.iloc[33,0]='Congo Republic'
gcont.iloc[34,0]='Democratic Republic of Congo'

In [9]:
# convert every name of country in standartized names with library cc
gcont['Country or Area']=cc.pandas_convert(series=gcont['Country or Area'],to='name_short')
# adding a column of CountryID converted in ISO3
gcont['CountryID']=cc.pandas_convert(series=gcont['Country or Area'],to='ISO3') 
gcont=gcont.drop(columns='UNSD M49 Codes')

In [10]:
#slight corrections of the countries that are not covered by cc
gcont.loc[len(gcont),:] = ["Yougoslavia (former)", "Southern Europe",'' ,"Europe","YUG"]
gcont.loc[len(gcont)] = ["USSR (former)", "Eastern Europe",'' ,"Europe","SUN"]
gcont.loc[len(gcont)] = ["Netherland Antilles (former)", "Caribbean", "Latin America and the Caribbean North America","Americas","ANT"]
gcont.loc[len(gcont)] = ["Zanzibar", "Eastern Africa",'' ,"Africa","EAZ"]

In [11]:
coll=list(gcont.columns)
coll[0]='Country'
gcont.columns=coll

In [12]:
gcont

,Country,Geographical subregion,Intermediary region,Continental region,CountryID
0,Algeria,Northern Africa,—,Africa,DZA
1,Egypt,Northern Africa,—,Africa,EGY
2,Libya,Northern Africa,—,Africa,LBY
3,Morocco,Northern Africa,—,Africa,MAR
4,Sudan,Northern Africa,—,Africa,SDN
...,...,...,...,...,...
249,Kosovo,Southern Europe,—,Europe,XKX
250,Yougoslavia (former),Southern Europe,,Europe,YUG
251,USSR (former),Eastern Europe,,Europe,SUN
252,Netherland Antilles (former),Caribbean,Latin America and the Caribbean North America,Americas,ANT


In [13]:
gcont.to_csv('dimgeo.csv')